In [115]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd
from sklearn.datasets import make_moons

#  Import nba data
nba_gamestats_df = pd.read_csv("Basketball_Data/curated_game_stats.csv")
print(nba_gamestats_df.head())

nba_advancedgame_df = pd.read_csv("Basketball_Data/Advanced_Game_Stats_2001-2003.csv")
nba_advancedgame_df.head()

   Unnamed: 0 Team        Date  PTS  Opp  FG  FGA 2PA  3P 3PA Opponent_Points
0           0  SAC  2023-02-24  176  LAC  65  111  70  18  41             175
1           1  LAC  2023-02-24  175  SAC  59   98  53  26  45             176
2           2  CHO  2022-01-26  158  IND  54   93  48  24  45             126
3           3  GSW  2023-04-09  157  POR  58   96  47  27  49             101
4           4  SAS  2022-02-25  157  WAS  57  105  76   9  29             153


,Rk,Team,Date,eFG%,Unnamed: 4,Opp,Result,ORtg,FTr,3PAr,TS%,eFG%.1,FT/FGA,ORtg.1,FTr.1,3PAr.1,TS%.1,eFG%.2,FT/FGA.1
0,1,DAL,2021-12-01,0.795,@,NOP,W 139-107,152.3,0.108,0.410,0.799,0.795,0.084,117.2,0.180,0.416,0.557,0.522,0.157
1,2,MIN,2022-12-18,0.787,NaN,CHI,W 150-126,147.9,0.184,0.494,0.798,0.787,0.149,124.2,0.273,0.432,0.639,0.614,0.205
2,3,MIN,2022-10-26,0.775,NaN,SAS,W 134-122,131.1,0.200,0.425,0.770,0.775,0.125,119.3,0.232,0.379,0.583,0.537,0.211
3,4,BRK,2022-12-21,0.768,NaN,GSW,W 143-113,139.8,0.190,0.393,0.785,0.768,0.167,110.5,0.301,0.398,0.601,0.566,0.229
4,5,LAC,2023-03-29,0.756,@,MEM,W 141-132,137.1,0.397,0.474,0.769,0.756,0.295,128.3,0.295,0.337,0.615,0.574,0.242


In [116]:
#Check types for nba game stats
nba_gamestats_df.dtypes

Unnamed: 0          int64
Team               object
Date               object
PTS                object
Opp                object
FG                 object
FGA                object
2PA                object
3P                 object
3PA                object
Opponent_Points    object
dtype: object

In [117]:
#Check types for nba advanced stats
nba_advancedgame_df.dtypes

Rk              int64
Team           object
Date           object
eFG%          float64
Unnamed: 4     object
Opp            object
Result         object
ORtg          float64
FTr           float64
3PAr          float64
TS%           float64
eFG%.1        float64
FT/FGA        float64
ORtg.1        float64
FTr.1         float64
3PAr.1        float64
TS%.1         float64
eFG%.2        float64
FT/FGA.1      float64
dtype: object

In [118]:
#Test matching dates needed for merge
nba_gamestats_df.loc[nba_gamestats_df["Date"] == nba_advancedgame_df["Date"][0]]

,Unnamed: 0,Team,Date,PTS,Opp,FG,FGA,2PA,3P,3PA,Opponent_Points
101,101,DAL,2021-12-01,139,NOP,57,83,49,18,34,107
650,650,MIL,2021-12-01,127,CHO,46,94,58,14,36,125
820,820,CHO,2021-12-01,125,MIL,43,87,36,21,51,127
924,924,SAC,2021-12-01,124,LAC,46,88,50,15,38,115
2107,2107,LAC,2021-12-01,115,SAC,47,100,69,9,31,124
2108,2108,WAS,2021-12-01,115,MIN,45,91,59,10,32,107
2278,2278,ATL,2021-12-01,114,IND,44,86,53,16,33,111
2279,2279,HOU,2021-12-01,114,OKC,40,79,42,15,37,110
2772,2772,CLE,2021-12-01,111,MIA,37,68,37,16,31,85
2773,2773,IND,2021-12-01,111,ATL,44,90,57,10,33,114


In [119]:
#Drop columns not needed, rename column
nba_advancedgame_df = nba_advancedgame_df.drop(["Result", "Rk", "FTr",	'3PAr',	'eFG%.1',	'FT/FGA',	'ORtg.1',	'FTr.1',	'3PAr.1',	'TS%.1',	'eFG%.2',	'FT/FGA.1'], axis=1)

nba_advancedgame_df.rename({
  "Unnamed: 4" : "Home/Away"
}, axis=1, inplace=True)


nba_advancedgame_df.head()

,Team,Date,eFG%,Home/Away,Opp,ORtg,TS%
0,DAL,2021-12-01,0.795,@,NOP,152.3,0.799
1,MIN,2022-12-18,0.787,NaN,CHI,147.9,0.798
2,MIN,2022-10-26,0.775,NaN,SAS,131.1,0.770
3,BRK,2022-12-21,0.768,NaN,GSW,139.8,0.785
4,LAC,2023-03-29,0.756,@,MEM,137.1,0.769


In [120]:
#Drop columns not needed
nba_gamestats_df = nba_gamestats_df.drop(["Unnamed: 0"], axis=1)

nba_gamestats_df.head()

,Team,Date,PTS,Opp,FG,FGA,2PA,3P,3PA,Opponent_Points
0,SAC,2023-02-24,176,LAC,65,111,70,18,41,175
1,LAC,2023-02-24,175,SAC,59,98,53,26,45,176
2,CHO,2022-01-26,158,IND,54,93,48,24,45,126
3,GSW,2023-04-09,157,POR,58,96,47,27,49,101
4,SAS,2022-02-25,157,WAS,57,105,76,9,29,153


In [121]:
complete_stats_df = pd.merge(nba_gamestats_df, nba_advancedgame_df, on=["Date", "Team", "Opp"])

In [122]:
complete_stats_df

,Team,Date,PTS,Opp,FG,FGA,2PA,3P,3PA,Opponent_Points,eFG%,Home/Away,ORtg,TS%
0,SAC,2023-02-24,176,LAC,65,111,70,18,41,175,0.667,@,136.9,0.696
1,LAC,2023-02-24,175,SAC,59,98,53,26,45,176,0.735,NaN,136.1,0.769
2,CHO,2022-01-26,158,IND,54,93,48,24,45,126,0.710,@,147.7,0.735
3,GSW,2023-04-09,157,POR,58,96,47,27,49,101,0.745,@,143.9,0.762
4,SAS,2022-02-25,157,WAS,57,105,76,9,29,153,0.586,@,130.7,0.633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,DAL,2022-03-09,77,NYK,27,86,42,6,44,107,0.349,NaN,86.3,0.397
4916,CHI,2021-11-22,77,IND,31,85,57,6,28,109,0.400,NaN,81.3,0.420
4917,SAC,2022-01-25,75,BOS,29,95,62,6,33,128,0.337,@,78.4,0.369
4918,NYK,2022-01-08,75,BOS,29,79,48,11,31,99,0.437,@,89.7,0.438


In [123]:
#Transform Dataframe
complete_stats_df=complete_stats_df.astype({
  'PTS' : 'float',
  'Opponent_Points' : 'float',
  'Home/Away': 'string',
  'FG': 'float',
  'FGA': 'float',
  '2PA': 'float',
  '3P': 'float',
  '3PA': 'float',
  'Team': 'string',
  'Opp': 'string'
})

complete_stats_df.dtypes

Team                string
Date                object
PTS                float64
Opp                 string
FG                 float64
FGA                float64
2PA                float64
3P                 float64
3PA                float64
Opponent_Points    float64
eFG%               float64
Home/Away           string
ORtg               float64
TS%                float64
dtype: object

In [124]:
#Add Column and convert home/away
complete_stats_df["Pts Difference"] =  complete_stats_df["PTS"] - complete_stats_df["Opponent_Points"]
complete_stats_df['Home/Away'] = complete_stats_df['Home/Away'].replace(['@', ''], ['Away', 'Home'])
complete_stats_df['Home/Away'].fillna(value="Home", inplace=True)

complete_stats_df.head()

,Team,Date,PTS,Opp,FG,FGA,2PA,3P,3PA,Opponent_Points,eFG%,Home/Away,ORtg,TS%,Pts Difference
0,SAC,2023-02-24,176.0,LAC,65.0,111.0,70.0,18.0,41.0,175.0,0.667,Away,136.9,0.696,1.0
1,LAC,2023-02-24,175.0,SAC,59.0,98.0,53.0,26.0,45.0,176.0,0.735,Home,136.1,0.769,-1.0
2,CHO,2022-01-26,158.0,IND,54.0,93.0,48.0,24.0,45.0,126.0,0.710,Away,147.7,0.735,32.0
3,GSW,2023-04-09,157.0,POR,58.0,96.0,47.0,27.0,49.0,101.0,0.745,Away,143.9,0.762,56.0
4,SAS,2022-02-25,157.0,WAS,57.0,105.0,76.0,9.0,29.0,153.0,0.586,Away,130.7,0.633,4.0


In [125]:
complete_stats_df.drop(["Date"], axis=1, inplace=True)

In [126]:
for_points_df = complete_stats_df.copy()

In [127]:
complete_stats_df.loc[complete_stats_df["Pts Difference"] >= 0, "Pts Difference"] = 1

In [128]:
complete_stats_df.loc[complete_stats_df["Pts Difference"] < 0, "Pts Difference"] = 0

In [129]:
complete_stats_df.head()

,Team,PTS,Opp,FG,FGA,2PA,3P,3PA,Opponent_Points,eFG%,Home/Away,ORtg,TS%,Pts Difference
0,SAC,176.0,LAC,65.0,111.0,70.0,18.0,41.0,175.0,0.667,Away,136.9,0.696,1.0
1,LAC,175.0,SAC,59.0,98.0,53.0,26.0,45.0,176.0,0.735,Home,136.1,0.769,0.0
2,CHO,158.0,IND,54.0,93.0,48.0,24.0,45.0,126.0,0.710,Away,147.7,0.735,1.0
3,GSW,157.0,POR,58.0,96.0,47.0,27.0,49.0,101.0,0.745,Away,143.9,0.762,1.0
4,SAS,157.0,WAS,57.0,105.0,76.0,9.0,29.0,153.0,0.586,Away,130.7,0.633,1.0


# Test Model for Win/Loss Prediction

In [220]:
# Split our preprocessed data into our features and target arrays
y = complete_stats_df['Pts Difference']
X = complete_stats_df.drop(["Team", "Opp",'Pts Difference'], axis= 1)


In [221]:
#One Hot Encoding
X = pd.get_dummies(X, columns=["Home/Away"])

In [222]:
X.head()

,PTS,FG,FGA,2PA,3P,3PA,Opponent_Points,eFG%,ORtg,TS%,Home/Away_Away,Home/Away_Home
0,176.0,65.0,111.0,70.0,18.0,41.0,175.0,0.667,136.9,0.696,1,0
1,175.0,59.0,98.0,53.0,26.0,45.0,176.0,0.735,136.1,0.769,0,1
2,158.0,54.0,93.0,48.0,24.0,45.0,126.0,0.710,147.7,0.735,1,0
3,157.0,58.0,96.0,47.0,27.0,49.0,101.0,0.745,143.9,0.762,1,0
4,157.0,57.0,105.0,76.0,9.0,29.0,153.0,0.586,130.7,0.633,1,0


In [223]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [224]:
# Create a StandardScaler instances
scaler = StandardScaler()


# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [229]:
#Instantiate Neural Network Model
nn_model = tf.keras.models.Sequential()

unit = 2 * X_train_scaled.shape[1]

#Dense Layers
nn_model.add(tf.keras.layers.Dense(units=unit, activation="relu", input_dim=X_train_scaled.shape[1]))
nn_model.add(tf.keras.layers.Dense(units=unit/4, activation="relu"))
# nn_model.add(tf.keras.layers.Dense(units=unit/6, activation="relu"))

#Output Layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


In [230]:
#Summary of Neural Network
nn_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_66 (Dense)            (None, 24)                312       
                                                                 
 dense_67 (Dense)            (None, 6)                 150       
                                                                 
 dense_68 (Dense)            (None, 1)                 7         
                                                                 
Total params: 469 (1.83 KB)
Trainable params: 469 (1.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [231]:
#Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#Train model
fit_model = nn_model.fit(X_train, y_train, epochs=200)

Epoch 1/200


116/116 [==============================] - 0s 641us/step - loss: 0.8696 - accuracy: 0.7694
Epoch 2/200
116/116 [==============================] - 0s 506us/step - loss: 0.1774 - accuracy: 0.9249
Epoch 3/200
116/116 [==============================] - 0s 527us/step - loss: 0.1604 - accuracy: 0.9312
Epoch 4/200
116/116 [==============================] - 0s 535us/step - loss: 0.1380 - accuracy: 0.9439
Epoch 5/200
116/116 [==============================] - 0s 509us/step - loss: 0.1248 - accuracy: 0.9482
Epoch 6/200
116/116 [==============================] - 0s 525us/step - loss: 0.1145 - accuracy: 0.9542
Epoch 7/200
116/116 [==============================] - 0s 511us/step - loss: 0.0967 - accuracy: 0.9661
Epoch 8/200
116/116 [==============================] - 0s 527us/step - loss: 0.0924 - accuracy: 0.9664
Epoch 9/200
116/116 [==============================] - 0s 538us/step - loss: 0.0985 - accuracy: 0.9588
Epoch 10/200
116/116 [==============================] - 0s 524us/step - loss: 0.0762 

In [232]:


model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.3858 - accuracy: 0.8667 - 73ms/epoch - 2ms/step
Loss: 0.3857569098472595, Accuracy: 0.8666666746139526


In [219]:
#Old copy of using one hot encoding on teams/opp teams
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 1.1498 - accuracy: 0.6634 - 72ms/epoch - 2ms/step
Loss: 1.1497598886489868, Accuracy: 0.6634146571159363


In [161]:
test_predict = X.loc[(X["Team_SAC"] == 1) & (X["Opp_LAC"] == 1)].copy()

In [165]:
scaler = StandardScaler()


# Fit the StandardScaler
X_scaler_test = scaler.fit(test_predict)

# Scale the data
X_predict = X_scaler_test.transform(X_train)
X_predict_test = X_scaler_test.transform(X_test)

In [177]:
nn_model.predict(X_predict).round()

29/29 [==============================] - 0s 555us/step


array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [180]:
pd.DataFrame(nn_model.predict(X_predict, batch_size=128).round()).value_counts()

 1/29 [>.............................] - ETA: 0s

29/29 [==============================] - 0s 560us/step


0.0    2166
1.0    1524
dtype: int64

# Test Model to Predict Point Differential


In [247]:
for_points_df

,Team,PTS,Opp,FG,FGA,2PA,3P,3PA,Opponent_Points,eFG%,Home/Away,ORtg,TS%,Pts Difference
0,SAC,176.0,LAC,65.0,111.0,70.0,18.0,41.0,175.0,0.667,Away,136.9,0.696,1.0
1,LAC,175.0,SAC,59.0,98.0,53.0,26.0,45.0,176.0,0.735,Home,136.1,0.769,-1.0
2,CHO,158.0,IND,54.0,93.0,48.0,24.0,45.0,126.0,0.710,Away,147.7,0.735,32.0
3,GSW,157.0,POR,58.0,96.0,47.0,27.0,49.0,101.0,0.745,Away,143.9,0.762,56.0
4,SAS,157.0,WAS,57.0,105.0,76.0,9.0,29.0,153.0,0.586,Away,130.7,0.633,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,DAL,77.0,NYK,27.0,86.0,42.0,6.0,44.0,107.0,0.349,Home,86.3,0.397,-30.0
4916,CHI,77.0,IND,31.0,85.0,57.0,6.0,28.0,109.0,0.400,Home,81.3,0.420,-32.0
4917,SAC,75.0,BOS,29.0,95.0,62.0,6.0,33.0,128.0,0.337,Away,78.4,0.369,-53.0
4918,NYK,75.0,BOS,29.0,79.0,48.0,11.0,31.0,99.0,0.437,Away,89.7,0.438,-24.0


In [265]:
y_points = for_points_df['Pts Difference']
X_points = for_points_df.drop(['Pts Difference', "Team", "Opp"], axis=1)
X_points = pd.get_dummies(X_points, columns=["Home/Away"])

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_points,y_points, random_state=42)

# Create a StandardScaler instances
scaler_p = StandardScaler()

# Fit the StandardScaler
X_scaler_p = scaler_p.fit(X_train_p)

# Scale the data
X_train_scaled_p = X_scaler_p.transform(X_train_p)
X_test_scaled_p = X_scaler_p.transform(X_test_p)


#Instantiate Neural Network Model
points_model = tf.keras.models.Sequential()

unit = X_train_scaled_p.shape[1]

#Dense Layers
points_model.add(tf.keras.layers.Dense(units=unit, activation="relu", input_dim=X_train_scaled_p.shape[1]))
points_model.add(tf.keras.layers.Dense(units=unit, activation="relu"))

#Output Layer
points_model.add(tf.keras.layers.Dense(units=1))

In [266]:
#Summary of Neural Network
points_model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_104 (Dense)           (None, 12)                156       
                                                                 
 dense_105 (Dense)           (None, 12)                156       
                                                                 
 dense_106 (Dense)           (None, 1)                 13        
                                                                 
Total params: 325 (1.27 KB)
Trainable params: 325 (1.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [268]:
#Compile the model
points_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

#Train model
fit_model_p = points_model.fit(X_train_p, y_train_p, epochs=200)

Epoch 1/200
116/116 [==============================] - 0s 541us/step - loss: 971.0218 - accuracy: 0.0041
Epoch 2/200
116/116 [==============================] - 0s 497us/step - loss: 206.5566 - accuracy: 0.0049
Epoch 3/200
116/116 [==============================] - 0s 512us/step - loss: 181.0760 - accuracy: 0.0060
Epoch 4/200
116/116 [==============================] - 0s 496us/step - loss: 103.9523 - accuracy: 0.0095
Epoch 5/200
116/116 [==============================] - 0s 529us/step - loss: 28.9863 - accuracy: 0.0100
Epoch 6/200
116/116 [==============================] - 0s 509us/step - loss: 11.3259 - accuracy: 0.0100
Epoch 7/200
116/116 [==============================] - 0s 521us/step - loss: 7.2620 - accuracy: 0.0103
Epoch 8/200
116/116 [==============================] - 0s 516us/step - loss: 6.4149 - accuracy: 0.0098
Epoch 9/200
116/116 [==============================] - 0s 516us/step - loss: 5.9044 - accuracy: 0.0103
Epoch 10/200
116/116 [==============================] - 0s 544u

In [255]:
model_loss_p, model_accuracy_p = points_model.evaluate(X_test_p,y_test_p,verbose=2)
print(f"Loss: {model_loss_p}, Accuracy: {model_accuracy_p}")

39/39 - 0s - loss: 0.0089 - accuracy: 0.0260 - 27ms/epoch - 704us/step
Loss: 0.008912559598684311, Accuracy: 0.02601625956594944


In [183]:
X_points


,PTS,FG,FGA,2PA,3P,3PA,Opponent_Points,eFG%,ORtg,TS%,...,Opp_PHI,Opp_PHO,Opp_POR,Opp_SAC,Opp_SAS,Opp_TOR,Opp_UTA,Opp_WAS,Home/Away_Away,Home/Away_Home
0,176.0,65.0,111.0,70.0,18.0,41.0,175.0,0.667,136.9,0.696,...,0,0,0,0,0,0,0,0,1,0
1,175.0,59.0,98.0,53.0,26.0,45.0,176.0,0.735,136.1,0.769,...,0,0,0,1,0,0,0,0,0,1
2,158.0,54.0,93.0,48.0,24.0,45.0,126.0,0.710,147.7,0.735,...,0,0,0,0,0,0,0,0,1,0
3,157.0,58.0,96.0,47.0,27.0,49.0,101.0,0.745,143.9,0.762,...,0,0,1,0,0,0,0,0,1,0
4,157.0,57.0,105.0,76.0,9.0,29.0,153.0,0.586,130.7,0.633,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,77.0,27.0,86.0,42.0,6.0,44.0,107.0,0.349,86.3,0.397,...,0,0,0,0,0,0,0,0,0,1
4916,77.0,31.0,85.0,57.0,6.0,28.0,109.0,0.400,81.3,0.420,...,0,0,0,0,0,0,0,0,0,1
4917,75.0,29.0,95.0,62.0,6.0,33.0,128.0,0.337,78.4,0.369,...,0,0,0,0,0,0,0,0,1,0
4918,75.0,29.0,79.0,48.0,11.0,31.0,99.0,0.437,89.7,0.438,...,0,0,0,0,0,0,0,0,1,0
